In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

The 2 websites are using json files as document model to create the ranking and to have the information of the universities. So by analysing the all the networks request and responses through `firefox` browser network tools, we find the url that responses the json files. 

In [3]:
tu_URL = 'https://www.topuniversities.com/sites/default/files/qs-rankings-data/357051.txt?_=1508326775940'
th_URL = 'https://www.timeshighereducation.com/sites/default/files/the_data_rankings/world_university_rankings_2018_limit0_369a9045a203e176392b9fb8f8c1cb2a.json'
json_tu = requests.get(tu_URL).json()
  

json_th = requests.get(th_URL).json()


Let's concentrate on the first university rankingwebsite, namely `Top University`. 

In [4]:
print(json_th.keys())

dict_keys(['data', 'subjects', 'locations', 'pillars'])


The `data` key contains a full description of each of the university of the ranking. 

In [5]:
data_th = json_th['data']
data_th[:10]

[{'aliases': 'University of Oxford',
  'location': 'United Kingdom',
  'member_level': '0',
  'name': 'University of Oxford',
  'nid': 468,
  'rank': '1',
  'rank_order': '10',
  'record_type': 'master_account',
  'scores_citations': '99.1',
  'scores_citations_rank': '15',
  'scores_industry_income': '63.7',
  'scores_industry_income_rank': '169',
  'scores_international_outlook': '95.0',
  'scores_international_outlook_rank': '24',
  'scores_overall': '94.3',
  'scores_overall_rank': '10',
  'scores_research': '99.5',
  'scores_research_rank': '1',
  'scores_teaching': '86.7',
  'scores_teaching_rank': '5',
  'stats_female_male_ratio': '46 : 54',
  'stats_number_students': '20,409',
  'stats_pc_intl_students': '38%',
  'stats_student_staff_ratio': '11.2',
  'subjects_offered': 'Archaeology,Art, Performing Arts & Design,Biological Sciences,Business & Management,Chemical Engineering,Chemistry,Civil Engineering,Computer Science,Economics & Econometrics,Electrical & Electronic Engineerin

In [6]:
data_tu = json_tu['data']
data_tu[:10]

[{'cc': 'US',
  'core_id': '410',
  'country': 'United States',
  'guide': '<a href="/where-to-study/north-america/united-states/guide" class="guide-link" target="_blank">United States</a>',
  'logo': '<img src="https://www.topuniversities.com/sites/default/files/massachusetts-institute-of-technology-mit_410_small_0.jpg" alt="Massachusetts Institute of Technology (MIT)  Logo">',
  'nid': '294850',
  'rank_display': '1',
  'region': 'North America',
  'score': '100',
  'stars': '6',
  'title': 'Massachusetts Institute of Technology (MIT)',
  'url': '/universities/massachusetts-institute-technology-mit'},
 {'cc': 'US',
  'core_id': '573',
  'country': 'United States',
  'guide': '<a href="/where-to-study/north-america/united-states/guide" class="guide-link" target="_blank">United States</a>',
  'logo': '<img src="https://www.topuniversities.com/sites/default/files/stanford-university_573_small_0.jpg" alt="Stanford University Logo">',
  'nid': '297282',
  'rank_display': '2',
  'region': 

Using those values, we will be able to extract the information demanded by the assignement: *name*, *rank*, *country* and *region* of the top 200 universities.

Note that the information about *number of faculty members* and *number of students* for each university is missing in the data. We treat this challenge separatly after.

In [7]:

# We limit ourself to the top 200 univerities as ask in the assignment.
top200_data = data_tu[:200]
tu_dict = [{'Name': u['title'], 'rank': u['rank_display'], 'country': u['country'], 'region': u['region']} for u in top200_data]

In order to get the additional information described above, we need to traverse the url of each of the top 200 universities. This step takes a large amoungt of time (compared to other process that we do, ~1min). Lukily, this step is done once. 



In [8]:
from datetime import datetime

unis_url = [{'name': u['title'], 'url': u['url']} for u in top200_data]
dictlist = []
base_url = "https://www.topuniversities.com"

# as numbers are of type string, this method is used to convert into int.
def nice_format(number):
    return int(number.strip('\n').replace(',',''))

start_time = datetime.now()


for url in unis_url[:200]:
    #print(url['name'])
    r = requests.get(base_url+url['url'])
    body = r.text
    soup = BeautifulSoup(body, 'html.parser')
    number = soup.find_all('div', class_='number')
    if len(number) is not 0:
        dictlist.append({'Name': url['name'], 'Total faculty members': nice_format(number[0].text), 'International faculty members'
                        : nice_format(number[1].text), 'Total number of students': nice_format(number[2].text), 'Total number of international students'
                        : nice_format(number[3].text)})
        
stop_time = datetime.now()

excetution_time = stop_time - start_time

print(' Time of the extraction of the extra information : {e}'.format(e=excetution_time))

 Time of the extraction of the extra information : 0:01:08.411917


Now we have the information needed and the following step consist on merging them. We use Dataframes to merge.

In [9]:
tu = pd.DataFrame(tu_dict)
tu

,Name,country,rank,region
0,Massachusetts Institute of Technology (MIT),United States,1,North America
1,Stanford University,United States,2,North America
2,Harvard University,United States,3,North America
3,California Institute of Technology (Caltech),United States,4,North America
4,University of Cambridge,United Kingdom,5,Europe
5,University of Oxford,United Kingdom,6,Europe
6,UCL (University College London),United Kingdom,7,Europe
7,Imperial College London,United Kingdom,8,Europe
8,University of Chicago,United States,9,North America
9,ETH Zurich - Swiss Federal Institute of Techno...,Switzerland,10,Europe


In [10]:
tu_add_info = pd.DataFrame(dictlist)
tu_add_info['Website'] = 'Top Universities'

tu = tu.merge(tu_add_info, how='outer', on='Name')
tu

,Name,country,rank,region,International faculty members,Total faculty members,Total number of international students,Total number of students,Website
0,Massachusetts Institute of Technology (MIT),United States,1,North America,1679.0,2982.0,3717.0,11067.0,Top Universities
1,Stanford University,United States,2,North America,2042.0,4285.0,3611.0,15878.0,Top Universities
2,Harvard University,United States,3,North America,1311.0,4350.0,5266.0,22429.0,Top Universities
3,California Institute of Technology (Caltech),United States,4,North America,350.0,953.0,647.0,2255.0,Top Universities
4,University of Cambridge,United Kingdom,5,Europe,2278.0,5490.0,6699.0,18770.0,Top Universities
5,University of Oxford,United Kingdom,6,Europe,2964.0,6750.0,7353.0,19720.0,Top Universities
6,UCL (University College London),United Kingdom,7,Europe,2554.0,6345.0,14854.0,31080.0,Top Universities
7,Imperial College London,United Kingdom,8,Europe,2071.0,3930.0,8746.0,16090.0,Top Universities
8,University of Chicago,United States,9,North America,635.0,2449.0,3379.0,13557.0,Top Universities
9,ETH Zurich - Swiss Federal Institute of Techno...,Switzerland,10,Europe,1886.0,2477.0,7563.0,19815.0,Top Universities


The Top university Dataframe table is ready. We wil do the same with Time Higher Education. 



In [11]:
#th_dict = [{'name': u['name'], 'rank': u['rank'], 'country' : u['location']} for u in data_th[:200]]
th_dict = []
for u in data_th[:200]:
    
    total_stu = float(u['stats_number_students'].replace(',', ''))
    inter_percent = float(u['stats_pc_intl_students'].strip('%'))
    student_staff_ratio = float(u['stats_student_staff_ratio'])
    
    th_dict.append(
        {'Name': u['name'],
         'rank': u['rank'], 
         'country' : u['location'],
         'Total number of students': int(total_stu), 
         'Total number of international students': int((total_stu / 100) * inter_percent),
         'Total faculty members': int(total_stu / student_staff_ratio),
         'Ratio faculty/students': 1 / student_staff_ratio,
         'Ratio international students': inter_percent / 100,
        }
    )
th = pd.DataFrame(th_dict)
th['Website'] = "Times Higher Education"
th['International faculty members'] = 'Unknown'
th

,Name,Ratio faculty/students,Ratio international students,Total faculty members,Total number of international students,Total number of students,country,rank,Website,International faculty members
0,University of Oxford,0.089286,0.38,1822,7755,20409,United Kingdom,1,Times Higher Education,Unknown
1,University of Cambridge,0.091743,0.35,1687,6436,18389,United Kingdom,2,Times Higher Education,Unknown
2,California Institute of Technology,0.153846,0.27,339,596,2209,United States,=3,Times Higher Education,Unknown
3,Stanford University,0.133333,0.22,2112,3485,15845,United States,=3,Times Higher Education,Unknown
4,Massachusetts Institute of Technology,0.114943,0.34,1284,3800,11177,United States,5,Times Higher Education,Unknown
5,Harvard University,0.112360,0.26,2283,5284,20326,United States,6,Times Higher Education,Unknown
6,Princeton University,0.120482,0.24,958,1909,7955,United States,7,Times Higher Education,Unknown
7,Imperial College London,0.087719,0.55,1390,8721,15857,United Kingdom,8,Times Higher Education,Unknown
8,University of Chicago,0.161290,0.25,2181,3381,13525,United States,9,Times Higher Education,Unknown
9,ETH Zurich – Swiss Federal Institute of Techno...,0.068493,0.38,1317,7308,19233,Switzerland,=10,Times Higher Education,Unknown


The 2 Dataframe are ready, we can answer the folowing assignment question.

Which are the best universities in term of: (a) ratio between faculty members and students, (b) ratio of international students?

For the Top university Website, the ratio between faculty members and students is computed as :

\begin{align}
\frac{Total\ faculty\ members}{Total\ number\ of\ student}
\end{align}

and the ratio of international students:
\begin{align}
\frac{Total\ number\ of\ international\ students}{Total\ number\ of\ student}
\end{align}

In [12]:
tu['Ratio faculty/students'] = tu['Total faculty members'].astype('float') / tu['Total number of students']
tu['Ratio international students'] = tu['Total number of international students'].astype('float') / tu['Total number of students']
tu

,Name,country,rank,region,International faculty members,Total faculty members,Total number of international students,Total number of students,Website,Ratio faculty/students,Ratio international students
0,Massachusetts Institute of Technology (MIT),United States,1,North America,1679.0,2982.0,3717.0,11067.0,Top Universities,0.269450,0.335863
1,Stanford University,United States,2,North America,2042.0,4285.0,3611.0,15878.0,Top Universities,0.269870,0.227422
2,Harvard University,United States,3,North America,1311.0,4350.0,5266.0,22429.0,Top Universities,0.193945,0.234785
3,California Institute of Technology (Caltech),United States,4,North America,350.0,953.0,647.0,2255.0,Top Universities,0.422616,0.286918
4,University of Cambridge,United Kingdom,5,Europe,2278.0,5490.0,6699.0,18770.0,Top Universities,0.292488,0.356899
5,University of Oxford,United Kingdom,6,Europe,2964.0,6750.0,7353.0,19720.0,Top Universities,0.342292,0.372870
6,UCL (University College London),United Kingdom,7,Europe,2554.0,6345.0,14854.0,31080.0,Top Universities,0.204151,0.477928
7,Imperial College London,United Kingdom,8,Europe,2071.0,3930.0,8746.0,16090.0,Top Universities,0.244251,0.543567
8,University of Chicago,United States,9,North America,635.0,2449.0,3379.0,13557.0,Top Universities,0.180645,0.249244
9,ETH Zurich - Swiss Federal Institute of Techno...,Switzerland,10,Europe,1886.0,2477.0,7563.0,19815.0,Top Universities,0.125006,0.381681


In [13]:
tu.columns

Index(['Name', 'country', 'rank', 'region', 'International faculty members',
       'Total faculty members', 'Total number of international students',
       'Total number of students', 'Website', 'Ratio faculty/students',
       'Ratio international students'],
      dtype='object')

In [14]:
tu.sort_values('Ratio faculty/students', ascending=False).head() # PROBLEMES
tu.sort_values('Ratio international students', ascending=False).head()

,Name,country,rank,region,International faculty members,Total faculty members,Total number of international students,Total number of students,Website,Ratio faculty/students,Ratio international students
189,Indian Institute of Science (IISc) Bangalore,India,190,Asia,4071.0,423.0,423.0,47.0,Top Universities,9.000000,9.000000
34,London School of Economics and Political Scien...,United Kingdom,35,Europe,687.0,1088.0,6748.0,9760.0,Top Universities,0.111475,0.691393
11,Ecole Polytechnique Fédérale de Lausanne (EPFL),Switzerland,12,Europe,1300.0,1695.0,5896.0,10343.0,Top Universities,0.163879,0.570047
7,Imperial College London,United Kingdom,8,Europe,2071.0,3930.0,8746.0,16090.0,Top Universities,0.244251,0.543567
198,Maastricht University,Netherlands,200,Europe,502.0,1277.0,8234.0,16385.0,Top Universities,0.077937,0.502533


We will answer to the same question but aggregates by counties. Note the use of lambsa function. Lamda function are handy to write and are easy to read.

In [15]:
tu.groupby('country').apply(lambda x: x.sort_values('Ratio faculty/students', ascending=False).head())

Name  \
country                                                                 
Argentina      74                   Universidad de Buenos Aires (UBA)   
Australia      19                  The Australian National University   
               46                        The University of Queensland   
               110                         The University of Adelaide   
               41                         The University of Melbourne   
               92                 The University of Western Australia   
Austria        153                               University of Vienna   
               184                    Vienna University of Technology   
Belgium        181                   Vrije Universiteit Brussel (VUB)   
               125                                   Ghent University   
               71                                           KU Leuven   
               152             Université catholique de Louvain (UCL)   
Brazil         120                          Universidade de São Paulo   
               182        Universidade Estadual de Campinas (Unicamp)   
Canada         139                                McMaster University   
               30                               University of Toronto   
               31                                   McGill University   
               90                               University of Alberta   
               50                      University of British Columbia   
Chile          137      Pontificia Universidad Católica de Chile (UC)   
               199                               Universidad de Chile   
China          24                                 Tsinghua University   
               39                                    Fudan University   
               96       University of Science and Technology of China   
               37                                   Peking University   
               86                                 Zhejiang University   
Denmark        116                    Technical University of Denmark   
               72                            University of Copenhagen   
               118                                  Aarhus University   
Finland        102                             University of Helsinki   
...                                                               ...   
South Korea    70   Pohang University of Science And Technology (P...   
               107                     Sungkyunkwan University (SKKU)   
               89                                    Korea University   
               36                           Seoul National University   
               105                                  Yonsei University   
Spain          186                     Universidad Autónoma de Madrid   
               155                           Universitat de Barcelona   
               195                  Universitat Autònoma de Barcelona   
Sweden         98                   KTH Royal Institute of Technology   
               132                  Chalmers University of Technology   
               78                                     Lund University   
               111                                 Uppsala University   
               194                               Stockholm University   
Switzerland    73                                University of Zurich   
               11     Ecole Polytechnique Fédérale de Lausanne (EPFL)   
               145                             University of Lausanne   
               9    ETH Zurich - Swiss Federal Institute of Techno...   
               97                                University of Geneva   
Taiwan         76                    National Taiwan University (NTU)   
               161                      National Tsing Hua University   
United Kingdom 5                                 University of Oxford   
               4                              University of Cambridge   
               7                              Impe

In [16]:
tu.groupby('country').apply(lambda x: x.sort_values('Ratio international students', ascending=False).head())

Name  \
country                                                                 
Argentina      74                   Universidad de Buenos Aires (UBA)   
Australia      41                         The University of Melbourne   
               19                  The Australian National University   
               110                         The University of Adelaide   
               49                            The University of Sydney   
               44     The University of New South Wales (UNSW Sydney)   
Austria        153                               University of Vienna   
               184                    Vienna University of Technology   
Belgium        181                   Vrije Universiteit Brussel (VUB)   
               152             Université catholique de Louvain (UCL)   
               71                                           KU Leuven   
               125                                   Ghent University   
Brazil         182        Universidade Estadual de Campinas (Unicamp)   
               120                          Universidade de São Paulo   
Canada         31                                   McGill University   
               30                               University of Toronto   
               90                               University of Alberta   
               151                             University of Waterloo   
               129                             Université de Montréal   
Chile          199                               Universidad de Chile   
               137      Pontificia Universidad Católica de Chile (UC)   
China          37                                   Peking University   
               39                                    Fudan University   
               86                                 Zhejiang University   
               24                                 Tsinghua University   
               114                                 Nanjing University   
Denmark        116                    Technical University of Denmark   
               118                                  Aarhus University   
               72                            University of Copenhagen   
Finland        138                                   Aalto University   
...                                                               ...   
South Korea    154                                 Hanyang University   
               89                                    Korea University   
               105                                  Yonsei University   
               107                     Sungkyunkwan University (SKKU)   
               36                           Seoul National University   
Spain          155                           Universitat de Barcelona   
               195                  Universitat Autònoma de Barcelona   
               186                     Universidad Autónoma de Madrid   
Sweden         78                                     Lund University   
               111                                 Uppsala University   
               132                  Chalmers University of Technology   
               98                   KTH Royal Institute of Technology   
               194                               Stockholm University   
Switzerland    11     Ecole Polytechnique Fédérale de Lausanne (EPFL)   
               97                                University of Geneva   
               9    ETH Zurich - Swiss Federal Institute of Techno...   
               148                                University of Basel   
               145                             University of Lausanne   
Taiwan         76                    National Taiwan University (NTU)   
               161                      National Tsing Hua University   
United Kingdom 34   London School of Economics and Political Scien...   
               7                              Imperial College London   
               6                      UCL (Univers

The aggregation is done with region.

In [17]:
tu.groupby('region').apply(lambda x: x.sort_values('Ratio faculty/students', ascending=False).head())

Name  \
region                                                                 
Africa        190                            University of Cape Town   
Asia          189       Indian Institute of Science (IISc) Bangalore   
              70   Pohang University of Science And Technology (P...   
              75                                   Tohoku University   
              35                                    Kyoto University   
              172       King Fahd University of Petroleum & Minerals   
Europe        5                                 University of Oxford   
              4                              University of Cambridge   
              7                              Imperial College London   
              116                    Technical University of Denmark   
              72                            University of Copenhagen   
Latin America 197  Instituto Tecnológico y de Estudios Superiores...   
              74                   Universidad de Buenos Aires (UBA)   
              121    Universidad Nacional Autónoma de México  (UNAM)   
              120                          Universidade de São Paulo   
              137      Pontificia Universidad Católica de Chile (UC)   
North America 3         California Institute of Technology (Caltech)   
              15                                     Yale University   
              16                            Johns Hopkins University   
              1                                  Stanford University   
              0          Massachusetts Institute of Technology (MIT)   
Oceania       19                  The Australian National University   
              46                        The University of Queensland   
              110                         The University of Adelaide   
              41                         The University of Melbourne   
              92                 The University of Western Australia   

                          country  rank         region  \
region                                                   
Africa        190    South Africa   191         Africa   
Asia          189           India   190           Asia   
              70      South Korea   =71           Asia   
              75            Japan   =76           Asia   
              35            Japan   =36           Asia   
              172    Saudi Arabia  =173           Asia   
Europe        5    United Kingdom     6         Europe   
              4    United Kingdom     5         Europe   
              7    United Kingdom     8         Europe   
              116         Denmark  =116         Europe   
              72          Denmark   =73         Europe   
Latin America 197          Mexico   199  Latin America   
              74        Argentina    75  Latin America   
              121          Mexico  =122  Latin America   
              120          Brazil   121  Latin America   
              137           Chile  =137  Latin America   
North America 3     United States     4  North America   
              15    United States    16  North America   
              16    United States    17  North America   
              1     United States     2  North America   
              0     United States     1  North America   
Oceania       19        Australia    20        Oceania   
              46        Australia   =47        Oceania   
              110       Australia  =109        Oceania   
              41        Australia   =41        Oceania   
              92        Australia   =93        Oceania   

                   International faculty members  Total faculty members  \
region                                                                    
Africa        190                          379.0                 1733.0   
Asia          189                         4071.0                  423.0   
              70                           113.0                  664.0   
              75                           264.

In [18]:
tu.groupby('region').apply(lambda x: x.sort_values('Ratio international students', ascending=False).head())

Name  \
region                                                                 
Africa        190                            University of Cape Town   
Asia          189       Indian Institute of Science (IISc) Bangalore   
              25                         The University of Hong Kong   
              48                        City University of Hong Kong   
              10   Nanyang Technological University, Singapore (NTU)   
              29   The Hong Kong University of Science and Techno...   
Europe        34   London School of Economics and Political Scien...   
              11     Ecole Polytechnique Fédérale de Lausanne (EPFL)   
              7                              Imperial College London   
              198                              Maastricht University   
              6                      UCL (University College London)   
Latin America 74                   Universidad de Buenos Aires (UBA)   
              197  Instituto Tecnológico y de Estudios Superiores...   
              199                               Universidad de Chile   
              137      Pontificia Universidad Católica de Chile (UC)   
              182        Universidade Estadual de Campinas (Unicamp)   
North America 47                          Carnegie Mellon University   
              0          Massachusetts Institute of Technology (MIT)   
              31                                   McGill University   
              17                                 Columbia University   
              30                               University of Toronto   
Oceania       41                         The University of Melbourne   
              19                  The Australian National University   
              110                         The University of Adelaide   
              49                            The University of Sydney   
              44     The University of New South Wales (UNSW Sydney)   

                          country  rank         region  \
region                                                   
Africa        190    South Africa   191         Africa   
Asia          189           India   190           Asia   
              25        Hong Kong    26           Asia   
              48        Hong Kong    49           Asia   
              10        Singapore    11           Asia   
              29        Hong Kong    30           Asia   
Europe        34   United Kingdom    35         Europe   
              11      Switzerland    12         Europe   
              7    United Kingdom     8         Europe   
              198     Netherlands   200         Europe   
              6    United Kingdom     7         Europe   
Latin America 74        Argentina    75  Latin America   
              197          Mexico   199  Latin America   
              199           Chile   201  Latin America   
              137           Chile  =137  Latin America   
              182          Brazil  =182  Latin America   
North America 47    United States   =47  North America   
              0     United States     1  North America   
              31           Canada    32  North America   
              17    United States    18  North America   
              30           Canada    31  North America   
Oceania       41        Australia   =41        Oceania   
              19        Australia    20        Oceania   
              110       Australia  =109        Oceania   
              49        Australia    50        Oceania   
              44        Australia    45        Oceania   

                   International faculty members  Total faculty members  \
region                                                                    
Africa        190                          379.0                 1733.0   
Asia          189                         4071.0                  423.0   
              25                          2085.0                 3012.0   
              48                          1027.

## Merge 

In order to merge the 2 universities, we need to have a commun unique information in the both side. We will use the names of the universities as unique identifier in the both table.

In [19]:
tu

,Name,country,rank,region,International faculty members,Total faculty members,Total number of international students,Total number of students,Website,Ratio faculty/students,Ratio international students
0,Massachusetts Institute of Technology (MIT),United States,1,North America,1679.0,2982.0,3717.0,11067.0,Top Universities,0.269450,0.335863
1,Stanford University,United States,2,North America,2042.0,4285.0,3611.0,15878.0,Top Universities,0.269870,0.227422
2,Harvard University,United States,3,North America,1311.0,4350.0,5266.0,22429.0,Top Universities,0.193945,0.234785
3,California Institute of Technology (Caltech),United States,4,North America,350.0,953.0,647.0,2255.0,Top Universities,0.422616,0.286918
4,University of Cambridge,United Kingdom,5,Europe,2278.0,5490.0,6699.0,18770.0,Top Universities,0.292488,0.356899
5,University of Oxford,United Kingdom,6,Europe,2964.0,6750.0,7353.0,19720.0,Top Universities,0.342292,0.372870
6,UCL (University College London),United Kingdom,7,Europe,2554.0,6345.0,14854.0,31080.0,Top Universities,0.204151,0.477928
7,Imperial College London,United Kingdom,8,Europe,2071.0,3930.0,8746.0,16090.0,Top Universities,0.244251,0.543567
8,University of Chicago,United States,9,North America,635.0,2449.0,3379.0,13557.0,Top Universities,0.180645,0.249244
9,ETH Zurich - Swiss Federal Institute of Techno...,Switzerland,10,Europe,1886.0,2477.0,7563.0,19815.0,Top Universities,0.125006,0.381681


In [20]:
#tu.set_index(keys=['Name'], inplace=True)

In [21]:
#th.set_index(keys = ['Name'], inplace=True)
#th

Is the indexes unique ?

In [22]:
print('Top university dataframe name index is unique : {b}'.format(b=tu.index.is_unique))
print('Times Higher Education dataframe name index is unique : {b}'.format(b=th.index.is_unique))

Top university dataframe name index is unique : True
Times Higher Education dataframe name index is unique : True


We merge the two dataframe with name index.

In [23]:
from difflib import SequenceMatcher

def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [24]:
tu_names = tu['Name'].tolist()
th_names = th['Name'].tolist()

for i in range(len(tu)):
    maxIndex = -1
    maxRatio = 0.0
    for j in range(len(th)):
        sim = similarity(tu_names[i], th_names[j])
        if sim > maxRatio and sim > 0.7:
            maxRatio = sim
            maxIndex = j
    if maxIndex != -1:
        th_names[maxIndex] = tu_names[i]
    
tu['Name'] = tu_names
th['Name'] = th_names

In [25]:
th['region'] = th.merge(tu, how='outer', on='Name')['region']

In [26]:
merged = tu.merge(th, how='outer')
merged = merged.groupby('Name').filter(lambda x: len(x) > 1)
merged = merged.set_index(['Name', 'country', 'region', 'Website']).sort_index(level=0)
merged

rank  \
Name                                               country        region        Website                        
Aalto University                                   Finland        Europe        Times Higher Education   190   
                                                                                Top Universities        =137   
Aarhus University                                  Denmark        Europe        Times Higher Education  =109   
                                                                                Top Universities         119   
Boston University                                  United States  North America Times Higher Education   =70   
                                                                                Top Universities          81   
Brown University                                   United States  North America Times Higher Education   =50   
                                                                                Top Universities          53   
Cardiff University                                 United Kingdom Europe        Times Higher Education  =162   
                                                                                Top Universities        =137   
Carnegie Mellon University                         United States  North America Times Higher Education    24   
                                                                                Top Universities         =47   
Chalmers University of Technology                  Netherlands    Europe        Times Higher Education    63   
                                                   Sweden         Europe        Top Universities         133   
City University of Hong Kong                       Hong Kong      Asia          Times Higher Education  =119   
                                                                                Top Universities          49   
Columbia University                                United States  North America Times Higher Education    14   
                                                                                Top Universities          18   
Dartmouth College                                  United States  North America Times Higher Education    89   
                                                                                Top Universities         169   
Duke University                                    United States  North America Times Higher Education    17   
                                                                                Top Universities         =21   
Durham University                                  United Kingdom Europe        Times Higher Education    97   
                                                                                Top Universities         =78   
ETH Zurich - Swiss Federal Institute of Technology Switzerland    Europe        Times Higher Education   =10   
                                                                                Top Universities          10   
Ecole Polytechnique                                France         Europe        Times Higher Education   115   
                                                                                Top Universities          59   
Ecole Polytechnique Fédérale de Lausanne (EPFL)    Switzerland    Europe        Times Higher Education   =38   
                                                                                Top Universities          12   
...                                                                                                      ...   
University of Waterloo                             Canada         North America Top Universities         152   
                                                   Switzerland    North America Times Higher Education   =95   
University of Wisconsin-Madison                    United States  North America Times Higher Education    43   
                                                                        

In [68]:
rk = merged['rank'].apply(lambda x: x.strip('=')).astype('int').groupby(by=['Name', 'country']).sum().sort_values().reset_index().drop('rank', axis=1)
rk.index += 1
rk.index.name = 'rank'
rk

,Name,country
rank,,
1,Indian Institute of Technology Bombay (IITB),United States
2,Stanford University,United States
3,Hokkaido University,United States
4,Massachusetts Institute of Technology (MIT),United States
5,University of Oxford,United Kingdom
6,University of Cambridge,United Kingdom
7,Imperial College London,United Kingdom
8,University of Chicago,United States
9,Korea University,United States


In [99]:
rk_cntr = rk.reset_index().drop('Name', axis=1).groupby('country').mean().sort_values('rank').reset_index().drop('rank', axis=1)
rk_cntr

,country
0,Singapore
1,Russia
2,Japan
3,United States
4,Australia
5,Hong Kong
6,Norway
7,Switzerland
8,South Korea
9,Canada


In [162]:
cntr = tu.merge(th, how='outer')[['Name', 'country']].drop_duplicates('Name').groupby('country').count()
cntr.reset_index(inplace=True)
cntr.columns = ['country', 'occurences']
cntr = cntr.sort_values(by='occurences', ascending=False).reset_index().drop('index', axis=1)
cntr.index += 1
cntr

,country,occurences
1,United States,61
2,United Kingdom,33
3,Germany,23
4,Netherlands,13
5,France,9
6,Australia,9
7,Japan,9
8,China,7
9,Switzerland,7
10,Sweden,7


In [174]:
#country_correl = cntr.sort_values('occurences', ascending=False).reset_index().drop(['occurences', 'index'], axis=1)
#country_correl.columns = ['country_occurences']
#country_correl['country_mean_rank'] = rk_cntr['country']
country_correl = cntr.reset_index().drop('occurences', axis=1)
country_correl.columns = [['index_occurences', 'country']]
res = country_correl.merge(rk_cntr.reset_index(), how='inner', on='country')
res.columns = [['index_occurences', 'country', 'index_rank']]
res.set_index('country', inplace=True)
res.corr()

,index_occurences,index_rank
index_occurences,1.000000,0.208775
index_rank,0.208775,1.000000
